# DeepDream with Inception V3 Model

####Preparations

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
BASE_PATH = '/content/drive/My Drive/Colab Notebooks/deep_dream/'
INPUT_PATH = os.path.join(BASE_PATH, 'input')
OUTPUT_PATH = os.path.join(BASE_PATH, 'output')

input_dir = os.listdir(INPUT_PATH)
output_dir = os.listdir(OUTPUT_PATH)

In [0]:
import numpy as np

import IPython.display as display
import PIL.Image

import tensorflow as tf

##Configuration & Model

In [0]:
print("input dir:\t" + str(list(enumerate(input_dir))))
print("output dir:\t" + str(list(enumerate(output_dir))))

In [0]:
# choose image from input dir list
item = 3

FILENAME = input_dir[item]

# max sidelength of input image
MAX_DIM = 500

# dreaming parameters
PARAMS = {"num_iterations": 20,
          "step_size": 0.01,
          "scaling": 0.7,
          "octaves": 4,
          "blend": 0.2}

PARAMS = dict(map(lambda kv: (kv[0], tf.constant(kv[1])), PARAMS.items()))

In [0]:
# choose output layer
layer_names = [
               'mixed1',
               #'mixed2',
               #'mixed3',
               #'mixed4', 
               #'mixed5',
               #'mixed6', 
               #'mixed7', 
               #'mixed8', 
               #'mixed9', 
               'mixed10'
               ]

base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

layers = [base_model.get_layer(name).output for name in layer_names]

model = tf.keras.Model(inputs=base_model.input, outputs=layers)

####IO Helper

In [0]:
def load(filename):
  '''
  filename  -- string

  returns   -- np.ndarray
  '''
  filepath = os.path.join(INPUT_PATH, filename)
  try:
    img = PIL.Image.open(filepath).convert("RGB")
  except:
    print("cannot to load " + filepath)
  if MAX_DIM:
      img.thumbnail((MAX_DIM, MAX_DIM))
  img = np.array(img)
  print("input shape:\t " + str(img.shape))
  return np.array(img, dtype=float)

def save(image, filename):
  '''
  img       -- np.ndarray
  filename  -- string

  returns   -- np.ndarray
  '''
  filepath = os.path.join(OUTPUT_PATH, filename)
  img = np.clip(image, 0, 255).astype('uint8')
  try:
    PIL.Image.fromarray(img).save(filepath)
    if os.path.exists(filepath):
      print("output saved: " + filename)
  except IOError:
    print("cannot save ", filepath)

def show(image):
  '''
  img       -- np.ndarray
  '''
  img = np.clip(image, 0, 255).astype('uint8')
  print("type, shape:\t" + str(type(img)) + ", " + str(img.shape))
  display.display(PIL.Image.fromarray(img))

####Image Processing

In [0]:
def preprocess(image):
  ''' The model was trained on normalized images.
  image     --  np.ndarray

  returns   --  tf.Tensor
  '''
  img = tf.keras.applications.inception_v3.preprocess_input(image)
  return tf.convert_to_tensor(img, dtype=tf.float32)

def deprocess(image):
  img = 255*(image + 1.0)/2.0
  return tf.cast(img, tf.uint8).numpy()

## future
def random_roll(image, maxroll):
  ''' To avoid tile seams.
  img       -- np.ndarray
  max       -- integer

  returns   -- integer, integer, np.ndarray
  '''
  dx, dy = np.random.randint(low=-maxroll, high=maxroll, size=(2,))
  img_rolled = np.roll(np.roll(image, dy, axis=0), dx, axis=1)
  return dx, dy, img_rolled

def un_roll(image, dx, dy):
  return np.roll(np.roll(image, -dy, axis=0), -dx, axis=1)

def gaussian_filter(image, kernel_size=11, sigma=5):
    def gauss_kernel(channels, kernel_size, sigma):
        ax = tf.range(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0)
        xx, yy = tf.meshgrid(ax, ax)
        kernel = tf.exp(-(xx ** 2 + yy ** 2) / (2.0 * sigma ** 2))
        kernel = kernel / tf.reduce_sum(kernel)
        kernel = tf.expand_dims(kernel, axis=-1)
        kernel = tf.tile(kernel, [1, 1, channels])
        return kernel

    gaussian_kernel = gauss_kernel(tf.shape(image)[-1], kernel_size, sigma)
    gaussian_kernel = tf.expand_dims(gaussian_kernel, axis=-1)

    try:
      blurred = tf.nn.depthwise_conv2d(input=image, filter=gaussian_kernel, strides=[1, 1, 1, 1],
                                  padding='SAME', data_format='NHWC')
    except:
      img_batch = tf.expand_dims(image, axis=0)
      blurred = tf.nn.depthwise_conv2d(input=img_batch, filter=gaussian_kernel, strides=[1, 1, 1, 1],
                                  padding='SAME', data_format='NHWC')
    return tf.squeeze(blurred)

## DeepDream Algorithm

In [0]:
def compute_cost(image):
  img = tf.expand_dims(image, axis=0)
  activations = model(img)

  layer_costs=[]
  for a in activations:
    a = tf.math.square(a)
    cost = tf.math.reduce_mean(a)
    layer_costs.append(cost)

  return tf.reduce_sum(layer_costs)

In [0]:
@tf.function(
    input_signature=(
      tf.TensorSpec(shape=[None, None, 3], dtype=tf.float32),
      tf.TensorSpec(shape=[], dtype=tf.float32),
      tf.TensorSpec(shape=[], dtype=tf.int32))
)
def step(image, step_size, i):
  with tf.GradientTape() as g:
    g.watch(image)
    J = compute_cost(image)
  
  s
  grad = g.gradient(J, image)
  '''
  N = tf.cast(PARAMS['num_iterations'], dtype=tf.float32)
  n = tf.cast(i, dtype=tf.float32)
  sigma = n * 4.0
  sigma =  sigma / N + 0.5
  grad_smooth1 = gaussian_filter(grad, sigma=sigma)
  grad_smooth2 = gaussian_filter(grad, sigma=sigma*2)
  grad_smooth3 = gaussian_filter(grad, sigma=sigma*0.5)

  grad = (grad_smooth1 + grad_smooth2 + grad_smooth3)
  '''
  step_size_scaled = step_size / (tf.math.reduce_std(grad) + tf.constant(1e-8))
  img = image + grad * step_size_scaled

  return J, img

In [0]:
def optimize_image(image, num_iterations, step_size):
  img = preprocess(image)

  num_iterations = tf.constant(num_iterations)
  step_size = tf.convert_to_tensor(step_size)

  for i in tf.range(num_iterations):
    print(".", end="")
    J, img = step(img, step_size, i)

  return img

In [0]:
def dream(image, num_iterations=10, octaves=4, scaling=0.7, blend=0.2, step_size=0.01):
  img = tf.constant(np.array(image), dtype=tf.float32)
  base_shape = tf.convert_to_tensor(image.shape[:-1], dtype=tf.int32)
  float_base_shape = tf.cast(base_shape, dtype=tf.float32)

  if octaves>0:
    img_blurred = img#gaussian_filter(img)
    #print(tf.shape(img_blurred))
    new_shape = tf.cast(float_base_shape*(scaling), dtype=tf.int32)
    img_scaled = tf.image.resize(img_blurred, new_shape).numpy()

    img_result = dream(img_scaled,
                       num_iterations=num_iterations, 
                       octaves=octaves-1, 
                       scaling=scaling, 
                       blend=blend, 
                       step_size=step_size)
  
    img_rescaled = tf.image.resize(img_result, base_shape)
    #img = tf.image.convert_image_dtype(img_rescaled, dtype=tf.float32)
    
    #print("img: " + str(tf.shape(img)))
    #print("image: " + str(tf.shape(image)))
    #print("blend: " + str(tf.shape(blend)))
    #print("img_rescaled: " + str(tf.shape(img_rescaled)))
    img = blend * img + (1.0 - blend) * img_rescaled
    #image = img_rescaled
    #print("image: " + str(tf.shape(image)))

  print("\noctave: " + str(octaves))
  img_result = optimize_image(img,
                              num_iterations=num_iterations,
                              step_size=step_size)
  
  return deprocess(img_result)

In [0]:
def run(filename, params):
  orig_image = load(filename)
  show(orig_image)

  #dx, dy, img_rolled = random_roll(orig_image, 512)
  #img_rolled_result = dream(image=img_rolled, **params)
  #img = un_roll(img_rolled_result, dx, dy)
  img = dream(image=orig_image, **params)
  show(img)
  save(img,  filename)

  return img

##Application

In [0]:
#%xmode Verbose
print("input file:\t" + str(FILENAME))
print("parameters:\t" + str(PARAMS))
print("layers:\t\t" + str(layer_names))
result = run(FILENAME, PARAMS)